In [24]:
import pandas as pd
import re
from dateutil.parser import parse
import io
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import requests
from datetime import datetime
import logging
import os
import gspread
import numpy as np
from leitura_arquivo_drive import GoogleSheetsManager

In [ ]:
#%pip install gspread==6.1.4

In [2]:
class GoogleAuthenticator:
    """Gerencia a autenticação do Google Sheets e Google Drive."""

    def __init__(self):
        self.sheets_client = None
        self.drive_service = None
        self.authenticate()

    def authenticate(self):
        """Autentica na API do Google Sheets e Google Drive."""
        try:
            scopes = [
                'https://www.googleapis.com/auth/spreadsheets',
                'https://www.googleapis.com/auth/drive'
            ]
            credentials_path = 'credentials/people-analytics-pipoca-agil-google-drive.json'
            credentials = Credentials.from_service_account_file(credentials_path, scopes=scopes)

            self.sheets_client = gspread.authorize(credentials)
            self.drive_service = build('drive', 'v3', credentials=credentials)

            logging.info("Autenticação bem-sucedida no Google Sheets e Google Drive.")
        except Exception as e:
            logging.error(f"Erro na autenticação: {e}")
            raise

In [3]:
class GoogleDriveManager:
    """Gerencia operações de upload de arquivos no Google Drive."""

    def __init__(self, drive_service):
        self.drive_service = drive_service
        self.Id_camada = {
            'raw': '1E6AEUGqRp3IJsWV4qAwMRJK_tMD7wDYT',
            'refined': '1tc3HQnG507HfyLvHqtZasb-La8GD98P0',
            'trusted': '1WJlq1C_uLq9J3Ta-lVAkQVv7AzblftsD'
        }

    def get_file_id(self, file_name, folder_id):
        """Busca o arquivo no Google Drive e retorna seu ID, se existir."""
        try:
            query = f"name='{file_name}' and '{folder_id}' in parents and trashed=false"
            results = self.drive_service.files().list(q=query, fields="files(id)").execute()
            files = results.get('files', [])
            return files[0]['id'] if files else None
        except Exception as e:
            logging.error(f"Erro ao buscar arquivo no Google Drive: {e}")
            return None
    
    def get_file_content(self, file_id):
        """Fetches the file content from Google Drive."""
        try:
            # Get file content (Excel file) from Google Drive
            request = self.drive_service.files().get_media(fileId=file_id)
            file_content = request.execute()
            return file_content
        except Exception as e:
            logging.error(f"Error while fetching file content: {e}")
            return None
        
    def get_file_data(self, file_id):
        """Busca o arquivo no Google Drive e retorna seu ID, se existir."""
        try:
            # Fetch the file content from Google Drive
            file_content = self.get_file_content(file_id)
            if not file_content:
                raise Exception("Failed to retrieve the file content.")

            # Use BytesIO to handle the file content in memory
            file_stream = io.BytesIO(file_content)

            # Read the Excel file from the in-memory byte stream
            df = pd.read_excel(file_stream)
            return df
        except Exception as e:
            logging.error(f"Error while reading Excel file: {e}")
            return None
        
        
    def download_existing_file(self, file_id, file_name):
        """Faz o download do arquivo existente no Google Drive."""
        try:
            request = self.drive_service.files().get_media(fileId=file_id)
            with open(file_name, "wb") as f:
                f.write(request.execute())
            logging.info(f"Arquivo {file_name} baixado com sucesso.")
        except Exception as e:
            logging.error(f"Erro ao baixar o arquivo: {e}")

    def save_data_to_layer(self, data, camada, relatorio):
        """Salva os dados na camada especificada do Google Drive."""
        try:
            folder_id = self.Id_camada[camada]
            file_name = f"{relatorio}.xlsx"
            file_id = self.get_file_id(file_name, folder_id)

            # Adiciona a timestamp de processamento
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            for row in data:
                row.append(timestamp)

            if file_id:
                self.download_existing_file(file_id, file_name)
                existing_df = pd.read_excel(file_name, header=None)
                new_df = pd.DataFrame(data)
                updated_df = pd.concat([existing_df, new_df], ignore_index=True)
                logging.info(f"Dados apendados à camada {camada}.")
            else:
                updated_df = pd.DataFrame(data)
                logging.info(f"Criando novo arquivo na camada {camada}.")

            # Salva o arquivo atualizado
            updated_df.to_excel(file_name, index=False, header=False)

            # Remove o arquivo antigo do Google Drive (se existir)
            if file_id:
                self.drive_service.files().delete(fileId=file_id).execute()
                logging.info(f"Arquivo antigo {file_name} removido do Google Drive.")

            # Faz upload do novo arquivo
            file_metadata = {
                'name': file_name,
                'parents': [folder_id]
            }
            media = MediaFileUpload(file_name, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
            uploaded_file = self.drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
            logging.info(f"Arquivo enviado para {camada}. ID: {uploaded_file.get('id')}")

            # Remove o arquivo local após o upload
            os.remove(file_name)

        except Exception as e:
            logging.error(f"Erro ao salvar o arquivo na camada {camada}: {e}")
            raise

def setup_logging():
    """Configura o logging para registrar informações e erros."""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
    )

In [ ]:
#tratamento da camada trusted para a camada refined

In [ ]:
auth = GoogleAuthenticator()
drive_manager = GoogleDriveManager(auth.drive_service)
folder_id = drive_manager.Id_camada['trusted']
relatorio = 'autoavaliacao.xlsx'  # Defina o relatório desejado
file_id = drive_manager.get_file_id(relatorio,folder_id)
autoavaliacao = drive_manager.get_file_data(file_id)
autoavaliacao = autoavaliacao.drop_duplicates() #para remover o cabeçalho caso ele se repita

In [ ]:
#transformar as perguntas em uma coluna
autoavaliacao = autoavaliacao.reset_index(drop=True)
#informações que são invariáveis e se repetem ao longo dos questionários
#coluna com a descrição do tipo de pergunta
tipo_perguntas = ['Quantitativa de 0 a 10','Quantitativa de 0 a 10','Quantitativa de 0 a 10','Quantitativa de 0 a 10',
                  'Quantitativa de 0 a 10','Quantitativa de 0 a 10','Quantitativa de 0 a 10,','Quantitativa de 0 a 10',
                  'sim/nao','Quantitativa de 0 a 10','Quantitativa de 0 a 10','Quantitativa de 0 a 10',
                  'Quantitativa de 0 a 10','Quantitativa de 0 a 10','Quantitativa de 0 a 10','Quantitativa de 0 a 10',
                  'Descritiva, texto de opinião']
#coluna com o tipo de dados das respostas
tipo_repostas = ['int','int','int','int','int','int','int','int','boolean','int','int','int','int','int','int','int',
                 'str']
perguntas = autoavaliacao.keys().tolist()[5:22]
#Cada entrevistado, que representa uma linha, que terá que ser transposta separadamente, transformado em dataframe e depois
#os dataframes serão concatenados verticalmente
all_df = []
for i_entrevistado in range(autoavaliacao.shape[0]):
    row = autoavaliacao.iloc[i_entrevistado].T
    row = row.apply(lambda x: int(x) if pd.notna(x) and isinstance(x, (np.float64, float)) else x) #transformar todos os campos float para integer
    fato_resposta = {'timestamp': 'int',
                             'dsEmailRespondente': row.iloc[1], #campo do email
                              'dsNomeRespondente': row.iloc[2], #campo do entrevistado    
                              'dsQualFuncaoDesempenha':row.iloc[3], #campo da função
                              'dsEquipeParticipante':row.iloc[4], #campo da equipe
                              'nmCadernoPergunta': 'Autoavaliação Pipoca Ágil (respostas)', #modificar para cada formulário
                              'dsTituloPergunta': perguntas, #lista com as perguntas
                              'dsTipoPergunta':tipo_perguntas, #lista com o tipo das perguntas
                              'dsResposta':row.iloc[5:22], #lista com as respotas
                              'dsDataType':tipo_repostas} #lista com os tipos das respostas
    df_fato_resposta = pd.DataFrame(fato_resposta).reset_index(drop=True) #criar um dataframe do dicionário fato_reposta
    all_df.append(df_fato_resposta) #unir todos os dataframes en uma lista
#para transformar as listas em um dataframe só
df_new = pd.concat(all_df)

C:\Users\Windows10\AppData\Local\Temp\ipykernel_14868\2948812946.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fato_resposta = {'timestamp': row[0],


In [20]:
df_new

,timestamp,dsEmailRespondente,dsNomeRespondente,dsQualFuncaoDesempenha,dsEquipeParticipante,nmCadernoPergunta,dsTituloPergunta,dsTipoPergunta,dsResposta,dsDataType
0,18/11/2024 20:35:23,sibelersoares77@gmail.com,Sibele Rodrigues Soares,Quality Assurance,Equipe Liderança de dados,Autoavaliação Pipoca Ágil (respostas),"4. Em uma escala de 1 a 10, como você avalia s...",Quantitativa de 0 a 10,6,int
1,18/11/2024 20:35:23,sibelersoares77@gmail.com,Sibele Rodrigues Soares,Quality Assurance,Equipe Liderança de dados,Autoavaliação Pipoca Ágil (respostas),"5. Em uma escala de 1 a 10, como você avalia o...",Quantitativa de 0 a 10,5,int
2,18/11/2024 20:35:23,sibelersoares77@gmail.com,Sibele Rodrigues Soares,Quality Assurance,Equipe Liderança de dados,Autoavaliação Pipoca Ágil (respostas),"6. Em uma escala de 1 a 10, como você avalia s...",Quantitativa de 0 a 10,5,int
3,18/11/2024 20:35:23,sibelersoares77@gmail.com,Sibele Rodrigues Soares,Quality Assurance,Equipe Liderança de dados,Autoavaliação Pipoca Ágil (respostas),"7. Em uma escala de 1 a 10, como você avalia s...",Quantitativa de 0 a 10,4,int
4,18/11/2024 20:35:23,sibelersoares77@gmail.com,Sibele Rodrigues Soares,Quality Assurance,Equipe Liderança de dados,Autoavaliação Pipoca Ágil (respostas),"8. Em uma escala de 1 a 10, como você avalia s...",Quantitativa de 0 a 10,4,int
5,18/11/2024 20:35:23,sibelersoares77@gmail.com,Sibele Rodrigues Soares,Quality Assurance,Equipe Liderança de dados,Autoavaliação Pipoca Ágil (respostas),"9. Em uma escala de 0 a 10, como você avalia s...",Quantitativa de 0 a 10,2,int
6,18/11/2024 20:35:23,sibelersoares77@gmail.com,Sibele Rodrigues Soares,Quality Assurance,Equipe Liderança de dados,Autoavaliação Pipoca Ágil (respostas),"10. Em uma escala de 0 a 10, como você avalia ...","Quantitativa de 0 a 10,",2,int
7,18/11/2024 20:35:23,sibelersoares77@gmail.com,Sibele Rodrigues Soares,Quality Assurance,Equipe Liderança de dados,Autoavaliação Pipoca Ágil (respostas),"11. Em uma escala de 0 a 10, como você avalia ...",Quantitativa de 0 a 10,2,int
8,18/11/2024 20:35:23,sibelersoares77@gmail.com,Sibele Rodrigues Soares,Quality Assurance,Equipe Liderança de dados,Autoavaliação Pipoca Ágil (respostas),12. Você acredita que seu time tem todos os pa...,sim/nao,Sim,boolean
9,18/11/2024 20:35:23,sibelersoares77@gmail.com,Sibele Rodrigues Soares,Quality Assurance,Equipe Liderança de dados,Autoavaliação Pipoca Ágil (respostas),"13. Em uma escala de 0 a 10, como você avalia ...",Quantitativa de 0 a 10,10,int


In [ ]:
#Verificar o id do folder onde está a fato_respostas, arquivo final
folder_id_refined = drive_manager.Id_camada['refined']
relatorio_refined = 'modelo_fato_respostas.xlsx'  # Defina o relatório desejado
file_id_refined = drive_manager.get_file_id(relatorio_refined,folder_id_refined)
fato_resposta = drive_manager.get_file_data(file_id_refined)
#adicionar as novas respostas à planilha fato_respostas
df_upload = pd.concat([fato_resposta,df_new])

In [ ]:
#Substituir o arquivo fato_respostas na camada trusted 
drive_manager.save_data_to_layer(df_upload,datetime.today(),relatorio_refined)


In [25]:
setup_logging()

auth = GoogleAuthenticator()
sheets_manager = GoogleSheetsManager(auth.sheets_client)
drive_manager = GoogleDriveManager(auth.drive_service)

relatorio = 'autoavaliacao'  # Defina o relatório desejado

In [26]:
sheet = sheets_manager.get_sheet(relatorio)  # Obtém a planilha
data = sheets_manager.get_data(sheet)  # Obtém os dados da planilha

In [27]:
data

[['Carimbo de data/hora',
  'Endereço de e-mail',
  '1. Seu nome completo:',
  '2. Qual função você desempenha no time?',
  '3. Qual equipe você participa?',
  '4. Em uma escala de 1 a 10, como você avalia sua capacidade de concluir as tarefas designadas durante a Sprint?  ',
  '5. Em uma escala de 1 a 10, como você avalia o suporte que recebeu no início das suas atividades?  ',
  '6. Em uma escala de 1 a 10, como você avalia sua clareza em relação ao progresso do time durante a Sprint?  ',
  '7. Em uma escala de 1 a 10, como você avalia sua participação nas Dailies e outras reuniões importantes durante a Sprint?',
  '8. Em uma escala de 1 a 10, como você avalia sua capacidade de concluir as tarefas planejadas durante a Sprint?  ',
  '9. Em uma escala de 0 a 10, como você avalia sua presença e participação nas reuniões (Dailies, Sprint Planning) ao longo da Sprint?',
  '10. Em uma escala de 0 a 10, como você avalia sua disponibilidade para cumprir com seus compromissos e tarefas durant